# Data Preparation

In this notebook, we will do feature extraction and data sampling, preparing the datasets for modelling step.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
aisles = pd.read_csv('data/raw/aisles.csv')
departments =  pd.read_csv('data/raw/departments.csv')
prior = pd.read_csv('data/raw/order_products__prior.csv')
train = pd.read_csv('data/raw/order_products__train.csv')
orders = pd.read_csv('data/raw/orders.csv')
products = pd.read_csv('data/raw/products.csv')

## Sampling and splitting
Due to computation limits, we decided to sample 10,000 users for our project. We will include all orders from these 10,000 users.

For each user, we use all history orders (in the prior dataset) as inputs to predict the target `reordered` in the train dataset (containing the most recent order).

Additionally, we will split the 10,000 users into training and testing. We will use 8000 users and all their data in the training, and the remaining 2000 users in the testing.

In [6]:
# Get sample
np.random.seed(32021)
sampled_users =np.random.choice(orders.user_id.unique(), size=10000, replace=False)
orders = orders[orders.user_id.isin(sampled_users)]

In [7]:
# Split users into training and testing
train_users, test_users = train_test_split(
    sampled_users, test_size=0.2, random_state=32021
)

## Feature Extraction
### Users' information
For each user, we will extract:
* `num_orders`: The total number of orders by the users, which will be the maximum of the `order_number` by that user.
* `avg_days_since_prior_order`: The average time between orders for the user.
* `num_items`: Total number of items the user has purchased across all orders (including reordered items).
* `user_reorder_prop`: Proportion of items that are reordered across all ordered items.

In [9]:
orders_prior = orders.merge(prior, on = 'order_id',how='inner')
orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2168274,2,prior,1,2,11,NaN,32792,1,0
1,2168274,2,prior,1,2,11,NaN,47766,2,0
2,2168274,2,prior,1,2,11,NaN,20574,3,0
3,2168274,2,prior,1,2,11,NaN,12000,4,0
4,2168274,2,prior,1,2,11,NaN,48110,5,0


In [10]:
users_info = orders.groupby('user_id').agg(
    num_orders = ('order_number', 'max'),
    avg_days_since_prior_order = ('days_since_prior_order','mean'),
).reset_index()

In [11]:
users_info_2 = orders_prior.groupby('user_id').agg(
    num_items = ('product_id','count'),
    user_reorder_prop = ('reordered','sum')
).reset_index()
users_info_2['user_reorder_prop'] = users_info_2['user_reorder_prop']/users_info_2['num_items']

In [12]:
users_info = users_info.merge(users_info_2, on = 'user_id',how='left')
users_info.head()

,user_id,num_orders,avg_days_since_prior_order,num_items,user_reorder_prop
0,2,15,16.285714,195,0.476923
1,14,14,21.230769,210,0.323810
2,66,7,28.833333,23,0.217391
3,89,21,9.200000,239,0.669456
4,94,10,16.333333,56,0.196429


### Products
For each product, we will extract these features:
* `product_total_orders`: Total number of orders across all users that include the product.
* `product_reorder_prop`: Proportion of all orders that include this product, and in which this product was reordered.
* `avg_add_to_cart_order`: The average order in which the product is added to cart across all orders that included this product.

In [14]:
products_info = prior.groupby('product_id').agg(
    product_total_orders = ('order_id','count'),
    product_reorder_prop = ('reordered','sum'),
    avg_add_to_cart_order = ('add_to_cart_order','mean')
).reset_index()
products_info['product_reorder_prop']=products_info['product_reorder_prop']/products_info['product_total_orders']
products_info.head()

,product_id,product_total_orders,product_reorder_prop,avg_add_to_cart_order
0,1,1852,0.613391,5.801836
1,2,90,0.133333,9.888889
2,3,277,0.732852,6.415162
3,4,329,0.446809,9.507599
4,5,15,0.600000,6.466667


In [15]:
# Merge extracted product features with aisles and departments
products_info=products_info.merge(products, on = 'product_id',how='left')
products_info.head()

,product_id,product_total_orders,product_reorder_prop,avg_add_to_cart_order,product_name,aisle_id,department_id
0,1,1852,0.613391,5.801836,Chocolate Sandwich Cookies,61,19
1,2,90,0.133333,9.888889,All-Seasons Salt,104,13
2,3,277,0.732852,6.415162,Robust Golden Unsweetened Oolong Tea,94,7
3,4,329,0.446809,9.507599,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,15,0.600000,6.466667,Green Chile Anytime Sauce,5,13


### User and Product Interactions
For each interaction between a user and a unique product, we extract:
* `times_bought_by_user`: Total number of orders by this user in which this product is included.
* `avg_user_product_position`: Average position among all user's orders in which this product was added.
* `last_order_number`: The most recent number order of this user where they last bought this product.

In [17]:
user_prod_info = orders_prior.groupby(['user_id','product_id']).agg(
    times_bought_by_user = ('order_id','count'),
    avg_user_product_position = ('add_to_cart_order','mean'),
    last_order_number = ('order_number', 'max')
).reset_index()
user_prod_info.head()

,user_id,product_id,times_bought_by_user,avg_user_product_position,last_order_number
0,2,23,1,12.00,8
1,2,79,1,3.00,13
2,2,1559,6,7.50,14
3,2,2002,4,10.25,11
4,2,2573,2,10.50,14


### Combine all extracted features

In [19]:
extracted_features = user_prod_info.merge(
    users_info, on='user_id', how='left'
).merge(
    products_info, on='product_id', how='left'
)
extracted_features.head()

,user_id,product_id,times_bought_by_user,avg_user_product_position,last_order_number,num_orders,avg_days_since_prior_order,num_items,user_reorder_prop,product_total_orders,product_reorder_prop,avg_add_to_cart_order,product_name,aisle_id,department_id
0,2,23,1,12.00,8,15,16.285714,195,0.476923,1068,0.448502,10.335206,Organic Turkey Burgers,49,12
1,2,79,1,3.00,13,15,16.285714,195,0.476923,3585,0.534449,10.440167,Wild Albacore Tuna No Salt Added,95,15
2,2,1559,6,7.50,14,15,16.285714,195,0.476923,6858,0.732429,8.384952,Cherry Pomegranate Greek Yogurt,120,16
3,2,2002,4,10.25,11,15,16.285714,195,0.476923,2832,0.541314,9.598870,"The \""World's Best\"" Veggie Burger",42,1
4,2,2573,2,10.50,14,15,16.285714,195,0.476923,1212,0.396040,10.398515,Garlic Pepper Ramen,66,6


## Prepare training and testing data
Here we will merge the extracted features from orders history with the target from the most recent order. The target is `reordered`, which answers: "Given a product that a user has purchased before, will this user reorder it in their next one?"

In [21]:
# Most recent orders
orders_train = orders.merge(train, on = 'order_id',how='inner')
orders_train.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1492625,2,train,15,1,11,30.0,22963,1,1
1,1492625,2,train,15,1,11,30.0,7963,2,1
2,1492625,2,train,15,1,11,30.0,16589,3,1
3,1492625,2,train,15,1,11,30.0,32792,4,1
4,1492625,2,train,15,1,11,30.0,41787,5,1


In [22]:
# Label 1 for products that are reordered in the most recent order
recent_ordered = orders_train[['user_id','product_id']].copy()
recent_ordered['reordered']= 1

In [23]:
data = extracted_features.merge(recent_ordered, on=['user_id', 'product_id'], how='left')
data['reordered'] = data['reordered'].fillna(0).astype(int)

#### Separate into training and testing set for export

In [25]:
train_data = data[data['user_id'].isin(train_users)]
test_data = data[data['user_id'].isin(test_users)]

In [26]:
train_data.to_csv('data/processed/train_data.csv')
test_data.to_csv('data/processed/test_data.csv')